In [10]:
import pandas as pd

'''
mapRating object contains elo rating dataframes and corresponding methods

Object fields ----------
    elo
    control
    hybrid
    assault
    escort
    directory
    ratings
Object methods ---------
   __init__
   updateRating()
   writeRating()
'''
class mapRating:
    def __init__(self):            
        # elo dataframes
        try:
            self.elo = pd.read_csv( "ELO.csv")
            self.control = pd.read_csv( "ControlELO.csv")
            self.hybrid = pd.read_csv( "HybridELO.csv")
            self.assault = pd.read_csv( "AssaultELO.csv")
            self.escort = pd.read_csv( "EscortELO.csv")
        except:
            print("Necessary files not present.")
            
         # dir
        self.directory = {  "Total"  :self.elo,
                            "Escort" :self.escort,
                            "Control":self.control,
                            "Hybrid" :self.hybrid,
                            "Assault":self.assault }
        
        # latest elo dicionary
        self.ratings = {"Total"  :self.elo.iloc[-1,:],
                        "Escort" :self.escort.iloc[-1,:],
                        "Control":self.control.iloc[-1,:],
                        "Hybrid" :self.hybrid.iloc[-1,:],
                        "Assault":self.assault.iloc[-1,:]}
    
    '''
    
    '''
    def updateRating(self, new_rating = [], key = "Total"):
        if key == "Total":
            #rating used to accumulate map ratings
            new_rating = [0 for x in range(20)]
            #iterate thru maps and get sum
            for typ in ["Escort", "Control", "Hybrid", "Assault"]:
                new_rating += self.ratings[typ]
            #average map ratings
            for j in range(len(new_rating)):
                new_rating[j] = new_rating[j]/4
            print(new_rating)
        #store new rating
        self.ratings[key] = new_rating
        return self.ratings[key]
    
    '''
    save new elo ratings in self.ratings to self.directory and 
        append the resulting directory to file
    '''
    def writeRating(self):
        for k in self.ratings:
            #update directory
            self.directory[k].append(self.ratings[k])
            #write (append) dataframes to CSV
            if k == "Total":
                self.directory[k].to_csv("ELO.csv", index=False, 
                                         mode='a', header = False)
            else:
                self.directory[k].to_csv(k+"ELO.csv", index=False, 
                                         mode = 'a', header = False)
        return

'''
method to update ELO values, adapted from 538's
    https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/
input:  rating -- current ELO values (expected dict)
        teamA, teamB -- key to teams to update
        sA, SB -- scored of respective teams
output: updated ELO values as dict
'''
def updateELO(rating, teamA, teamB, sA, sB):
    if (sA > sB):
        sA = 1
    elif( sA < sB):
        sA = 0
    else:
        sA = .5
    
    sB = 1 - sA
    
    qA = 10**(rating[teamA] / 400)
    qB = 10**(rating[teamB] / 400)
    
    eA = qA/(qA + qB)
    eB = qB/(qA + qB)
    
    rating[teamA] = rating[teamA] + 20 * (sA - eA)
    rating[teamB] = rating[teamB] + 20 * (sB - eB)
    
    return rating

'''
mapping dictionary linking long form team name to team code
'''
TEAM = dict()
TEAM["Atlanta Reign"] = "ATL"
TEAM["Boston Uprising"] = "BOS"
TEAM["Chengdu Hunters"] = "CHD"
TEAM["Dallas Fuel"] = "DAL"
TEAM["Florida Mayhem"] = "FLA"
TEAM["Guangzhou Charge"] = "GZC"
TEAM["Hangzhou Spark"] = "HZS"
TEAM["Houston Outlaws"] = "HOU"
TEAM["London Spitfire"] = "LDN"
TEAM["Los Angeles Gladiators"] = "GLA"
TEAM["Los Angeles Valiant"] = "VAL"
TEAM["New York Excelsior"] = "NYE"
TEAM["Paris Eternal"] = "PAR"
TEAM["Philadelphia Fusion"] = "PHI"
TEAM["San Francisco Shock"] = "SFS"
TEAM["Seoul Dynasty"] = "SEO"
TEAM["Shanghai Dragons"] = "SHD"
TEAM["Toronto Defiant"] = "TOR"
TEAM["Vancouver Titans"] = "VAN"
TEAM["Washington Justice"] = "WAS"

'''
mapping dictionary linking map name to game type
'''
MP = dict()
MP["Ilios"] = "Control"
MP["Busan"] = "Control"
MP["Nepal"] = "Control"
MP["Oasis"] = "Control"
MP["Lijiang Tower"] = "Control"

MP["Havana"] = "Escort"
MP["Rialto"] = "Escort"
MP["Watchpoint: Gibraltar"] = "Escort"
MP["Dorado"] = "Escort"
MP["Junkertown"] = "Escort"
MP["Route 66"] = "Escort"

MP["Horizon Lunar Colony"] = "Assault"
MP["Hanamura"] = "Assault"
MP["Temple of Anubis"] = "Assault"
MP["Volskaya Industries"] = "Assault"
MP["Paris"] = "Assault"

MP["Eichenwalde"] = "Hybrid"
MP["King's Row"] = "Hybrid"
MP["Blizzard World"] = "Hybrid"
MP["Numbani"] = "Hybrid"
MP["Hollywood"] = "Hybrid"

In [9]:
df = pd.read_csv("match_map_stats.csv")

elo = mapRating()

df = df[df["map_round"] == 1]
df = df[['round_end_time','stage','match_id', 'map_winner', 'map_loser', 'map_name',
         'winning_team_final_map_score', 'losing_team_final_map_score',
         'attacker', 'defender']]

cur = df.iloc[0,2]
day = df.iloc[0,0].split(' ')[0]

for i, (d, st, mid, mwin, mlos, mapn, wscore, lscore, atk, dfn) in df.iterrows():   
    # new match
    if cur != mid:
        cur = mid
        #write update total rating
        elo.updateRating()

        cday = d.split(' ')[0]
        if day != cday:
            day = cday
            elo.writeRating()
            
            if i > 50:
                break
        
    if mwin == "draw":
        mwin = atk
        mlos = dfn
    
    tmp = updateELO(elo.ratings[MP[mapn]], TEAM[mwin], TEAM[mlos], wscore, lscore)
    elo.updateRating(tmp,MP[mapn])
    #print(MP[mapn],elo.ratings[MP[mapn]])

#elo.ratings

Escort ATL    1500
BOS    1500
CHD    1500
DAL    1500
FLA    1500
GLA    1500
GZC    1500
HOU    1500
HZS    1500
LDN    1500
NYE    1500
PAR    1500
PHI    1500
SFS    1490
SEO    1500
SHD    1500
TOR    1500
VAL    1510
VAN    1500
WAS    1500
Name: 0, dtype: int64
Assault ATL    1500
BOS    1500
CHD    1500
DAL    1500
FLA    1500
GLA    1500
GZC    1500
HOU    1500
HZS    1500
LDN    1500
NYE    1500
PAR    1500
PHI    1500
SFS    1490
SEO    1500
SHD    1500
TOR    1500
VAL    1510
VAN    1500
WAS    1500
Name: 0, dtype: int64
Control ATL    1500
BOS    1500
CHD    1500
DAL    1500
FLA    1500
GLA    1500
GZC    1500
HOU    1500
HZS    1500
LDN    1500
NYE    1500
PAR    1500
PHI    1500
SFS    1490
SEO    1500
SHD    1500
TOR    1500
VAL    1510
VAN    1500
WAS    1500
Name: 0, dtype: int64
Hybrid ATL    1500
BOS    1500
CHD    1500
DAL    1500
FLA    1500
GLA    1500
GZC    1500
HOU    1500
HZS    1500
LDN    1500
NYE    1500
PAR    1500
PHI    1500
SFS    1490
SEO    1500
SHD 

In [7]:
df = pd.read_csv("match_map_stats.csv")
df.head()

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,map_round,...,team_one_name,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score
0,1/11/2018 0:12,1/11/2018 0:20,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,1,...,Los Angeles Valiant,San Francisco Shock,75.615051,0.000000,0.000000,240.000000,NaN,NaN,2,0
1,1/11/2018 0:22,1/11/2018 0:27,Overwatch League - Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,2,...,Los Angeles Valiant,San Francisco Shock,75.649597,75.615051,125.750572,0.000000,NaN,NaN,3,2
2,1/11/2018 0:34,1/11/2018 0:38,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,1,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,250.492004,240.000000,NaN,NaN,2,0
3,1/11/2018 0:40,1/11/2018 0:44,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,2,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,225.789032,250.492004,NaN,NaN,2,2
4,1/11/2018 0:46,1/11/2018 0:49,Overwatch League - Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,3,...,Los Angeles Valiant,San Francisco Shock,0.000000,0.000000,36.396057,250.492004,NaN,NaN,4,2


In [34]:
elo.ratings['Control'].shape

(21,)

In [9]:
elo.directory["Control"]

,ATL,BOS,CHD,DAL,FLA,GLA,GZC,HOU,HZS,LDN,NYE,PAR,PHI,SFS,SEO,SHD,TOR,VAL,VAN,WAS
0,1542,1364,1495,1444,1300,1461,1521,1453,1517,1435,1517,1377,1411,1581,1508,1430,1426,1480,1603,1429
